In [1]:
import os

import pandas as pd
import tiktoken

from graphrag.config.enums import ModelType
from graphrag.config.models.language_model_config import LanguageModelConfig
from graphrag.language_model.manager import ModelManager
from graphrag.query.indexer_adapters import (
    read_indexer_communities,
    read_indexer_entities,
    read_indexer_reports,
)
from graphrag.query.structured_search.global_search.community_context import (
    GlobalCommunityContext,
)
from graphrag.query.structured_search.global_search.search import GlobalSearch

/opt/conda/envs/ai-agents/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# feedback why should we pass the type to both config and the chat_model etc?

from graphrag.config.enums import ModelType, AuthType
from graphrag.config.models.language_model_config import LanguageModelConfig
from graphrag.language_model.manager import ModelManager
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

api_key = os.getenv("GRAPHRAG_API_KEY")
llm_model = os.getenv("GRAPHRAG_LLM_MODEL")
embedding_model = os.getenv("GRAPHRAG_EMBEDDING_MODEL")

config = LanguageModelConfig(
    api_key=api_key,
    auth_type=AuthType.APIKey, 
    type=ModelType.AzureOpenAIChat,
    model=llm_model,
    deployment_name=llm_model,
    max_retries=20,
    api_base= os.getenv("GRAPHRAG_API_BASE"),
    api_version="2024-02-15-preview"
)
model = ModelManager().get_or_create_chat_model(
    name="local_search",
    model_type=ModelType.AzureOpenAIChat,
    config=config,
)

token_encoder = tiktoken.encoding_for_model(llm_model)



In [3]:
INPUT_DIR = "./output"

COMMUNITY_REPORT_TABLE = "community_reports"
ENTITY_TABLE = "entities"
COMMUNITY_TABLE = "communities"
COMMUNITY_LEVEL = 2

In [4]:
community_df = pd.read_parquet(f"{INPUT_DIR}/{COMMUNITY_TABLE}.parquet")
entity_df = pd.read_parquet(f"{INPUT_DIR}/{ENTITY_TABLE}.parquet")
report_df = pd.read_parquet(f"{INPUT_DIR}/{COMMUNITY_REPORT_TABLE}.parquet")

communities = read_indexer_communities(community_df, report_df)
reports = read_indexer_reports(report_df, community_df, COMMUNITY_LEVEL)
entities = read_indexer_entities(entity_df, community_df, COMMUNITY_LEVEL)

print(f"Total report count: {len(report_df)}")
print(
    f"Report count after filtering by community level {COMMUNITY_LEVEL}: {len(reports)}"
)

report_df.head()

Total report count: 270
Report count after filtering by community level 2: 266


,id,human_readable_id,community,level,parent,children,title,summary,full_content,rank,rating_explanation,findings,full_content_json,period,size
0,4a1b360a941242928932582e582b2d8a,266,266,3,167,[],Microsoft Fabric Ecosystem,The Microsoft Fabric community encompasses a r...,# Microsoft Fabric Ecosystem\n\nThe Microsoft ...,8.5,The impact severity rating is high due to the ...,[{'explanation': 'Microsoft Fabric is deeply i...,"{\n ""title"": ""Microsoft Fabric Ecosystem"",\...",2025-05-17,252
1,9f6f2f0256e947afb79201c599dcd51b,267,267,3,167,[],Fabric Community Conference in Las Vegas,The Fabric Community Conference is an annual e...,# Fabric Community Conference in Las Vegas\n\n...,6.0,The impact severity rating is moderate due to ...,[{'explanation': 'The Fabric Community Confere...,"{\n ""title"": ""Fabric Community Conference i...",2025-05-17,2
2,f7d208f8d7d5448d913ae0f36140fe54,268,268,3,233,[],Microsoft Fabric Data Management Community,The community centers around Microsoft Fabric'...,# Microsoft Fabric Data Management Community\n...,7.5,The impact severity rating is high due to the ...,[{'explanation': 'One Lake serves as a pivotal...,"{\n ""title"": ""Microsoft Fabric Data Managem...",2025-05-17,8
3,b4b3237867a14c91b79c92cd324f7657,269,269,3,233,[],Delta Lake and Associated Technologies,"The community centers around Delta Lake, an op...",# Delta Lake and Associated Technologies\n\nTh...,8.5,The impact severity rating is high due to the ...,[{'explanation': 'Delta Lake serves as a core ...,"{\n ""title"": ""Delta Lake and Associated Tec...",2025-05-17,3
4,41cc1742a4bf40c3ad88930d6cb5a8c3,153,153,2,19,[],Azure OpenAI and Its Ecosystem,"The community centers around Azure OpenAI, a s...",# Azure OpenAI and Its Ecosystem\n\nThe commun...,8.5,The impact severity rating is high due to the ...,[{'explanation': 'Azure OpenAI is a comprehens...,"{\n ""title"": ""Azure OpenAI and Its Ecosyste...",2025-05-17,3


In [5]:
context_builder = GlobalCommunityContext(
    community_reports=reports,
    communities=communities,
    entities=entities,  # default to None if you don't want to use community weights for ranking
    token_encoder=token_encoder,
)

### Parameter descriptions for Global Search
- **model**: The LLM/chat model used for generating intermediate and final responses.
- **context_builder**: Prepares and batches context data from community reports for the map-reduce process.
- **token_encoder**: Optional token encoder (e.g., from tiktoken) for managing token limits.
- **map_system_prompt**: Prompt template for the "map" phase (intermediate response generation). If not provided, uses default.
- **reduce_system_prompt**: Prompt template for the "reduce" phase (final aggregation). If not provided, uses default.
- **response_type**: Specifies the format of the final answer (e.g., "multiple paragraphs").
- **allow_general_knowledge**: If True, allows general knowledge (outside the dataset) to be included in the response.
- **general_knowledge_inclusion_prompt**: Custom prompt for including general knowledge.
- **json_mode**: If True, expects and parses JSON-formatted LLM responses in the map phase.
- **callbacks**: List of callback objects for logging or monitoring query execution.
- **max_data_tokens**: Maximum tokens allowed for context data (controls batching).
- **map_llm_params**: Dict of extra parameters to pass to the map-phase LLM calls (e.g., temperature, max_tokens).
- **reduce_llm_params**: Dict of extra parameters for the reduce-phase LLM call.
- **map_max_length**: Maximum number of tokens/words in a map-phase LLM response.
- **reduce_max_length**: Maximum number of tokens/words in a reduce-phase LLM response.
- **context_builder_params**: Dict of parameters to pass to the context builder.
- **concurrent_coroutines**: Number of parallel coroutines to use during the map phase.

In [6]:
### Testing the callback

from graphrag.callbacks.query_callbacks import QueryCallbacks

class PrintMapResultCallback(QueryCallbacks):
    def on_map_result(self, map_result, **kwargs):
        print("Intermediate map result:", map_result)

    # Optionally override other methods if needed
    # def on_reduce_result(self, reduce_result, **kwargs):
    #     print("Reduce result:", reduce_result)

# Instantiate your callback(s)
callbacks = [PrintMapResultCallback()]

In [7]:
context_builder_params = {
    "use_community_summary": False,  # False means using full community reports. True means using community short summaries.
    "shuffle_data": True,
    "include_community_rank": True,
    "min_community_rank": 0,
    "community_rank_name": "rank",
    "include_community_weight": True,
    "community_weight_name": "occurrence weight",
    "normalize_community_weight": True,
    "max_tokens": 20_000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 5000)
    "context_name": "Reports",
}

map_llm_params = {
    "max_tokens": 1000,
    "temperature": 0.0,
    "response_format": {"type": "json_object"},
}

reduce_llm_params = {
    "max_tokens": 5000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 1000-1500)
    "temperature": 0.0,
}

In [8]:
search_engine = GlobalSearch(
    model=model,
    callbacks=callbacks,
    context_builder=context_builder,
    token_encoder=token_encoder,
    max_data_tokens=12_000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 5000)
    map_llm_params=map_llm_params,
    reduce_llm_params=reduce_llm_params,
    allow_general_knowledge=False,  # set this to True will add instruction to encourage the LLM to incorporate general knowledge in the response, which may increase hallucinations, but could be useful in some use cases.
    json_mode=True,  # set this to False if your LLM model does not support JSON mode.
    context_builder_params=context_builder_params,
    concurrent_coroutines=32,
    response_type="multiple paragraphs",  # free form text describing the response type and format, can be anything, e.g. prioritized list, single paragraph, multiple paragraphs, multiple-page report
)

In [9]:
context = await context_builder.build_context(
    query="how do you do shortcuts?",
    **context_builder_params
)
print(context)

ContextBuilderResult(context_chunks=['id|title|occurrence weight|content|rank\n7|Microsoft Fabric and Power BI Ecosystem|0.9808917197452229|"# Microsoft Fabric and Power BI Ecosystem\n\nThe community centers around Microsoft Fabric and its associated tools, including Power BI, Azure Analysis Services, and Data Mart. These entities are interconnected, facilitating data storage, analytics, and reporting capabilities, which are essential for organizations looking to leverage data for strategic decision-making.\n\n## Microsoft Fabric as a foundational platform\n\nMicrosoft Fabric serves as the core platform that integrates various data services, including Data Mart and Azure Analysis Services. This integration allows organizations to manage and analyze data efficiently, providing a comprehensive solution for data-driven decision-making. The ability to access and utilize multiple analytics tools within a single framework enhances operational efficiency and supports complex data workflows. O

In [ ]:
result = await search_engine.search("how do you do shortcuts?")
print(result.response)

In [ ]:
# inspect the data used to build the context for the LLM responses
result.context_data["reports"]

,id,title,occurrence weight,content,rank
0,7,Microsoft Fabric and Power BI Ecosystem,0.980892,# Microsoft Fabric and Power BI Ecosystem\n\nT...,8.5
1,155,Microsoft Fabric Community: Data Processing an...,0.356688,# Microsoft Fabric Community: Data Processing ...,8.5
2,144,Center of Excellence and Business Units Collab...,0.114650,# Center of Excellence and Business Units Coll...,7.5
3,152,Data Governance Community,0.076433,# Data Governance Community\n\nThe Data Govern...,8.5
4,154,Azure OpenAI Service and Its Ecosystem,0.050955,# Azure OpenAI Service and Its Ecosystem\n\nTh...,7.5
...,...,...,...,...,...
261,10,Microsoft Fabric Data Ecosystem,0.267516,# Microsoft Fabric Data Ecosystem\n\nThe Micro...,8.5
262,159,Copilot in Fabric and Generative AI Community,0.038217,# Copilot in Fabric and Generative AI Communit...,7.5
263,188,Content Delivery Solutions Community,0.006369,# Content Delivery Solutions Community\n\nThe ...,7.5
264,68,Data Governance Program Community,0.006369,# Data Governance Program Community\n\nThe Dat...,7.5


In [ ]:
# inspect number of LLM calls and tokens
print(
    f"LLM calls: {result.llm_calls}. Prompt tokens: {result.prompt_tokens}. Output tokens: {result.output_tokens}."
)

LLM calls: 26. Prompt tokens: 207759. Output tokens: 1429.
